# 02k – 12. Experiment: Post-Bugfix: Grid-Search min_duration_on × min_duration_off

## Kontext: Warum dieser Grid-Search?

`02j_` zeigte, dass der Bugfix in `segmentation.py` die WER *verschlechtert* hat (beam=12, len=20: +0.029). Die korrigierte
`min_duration_off`-Einstellung ändert, wie Sprachpausen verarbeitet werden.
Ziel dieses Notebooks ist es, durch gezielte Parametersuche die WER
unter das Vor-Bugfix-Niveau zu senken.

### Der Bug (zur Erinnerung)

```python
# FEHLERHAFT: min_duration_off las den Fallback von min_duration_ON
min_duration_off_frames = int(parameters.get("min_duration_off",
    CENTRAL_ASD_CHUNKING_PARAMETERS["min_duration_on"]) * 25)

# KORREKT:
min_duration_off_frames = int(parameters.get("min_duration_off",
    CENTRAL_ASD_CHUNKING_PARAMETERS["min_duration_off"]) * 25)
```

### Parameter-Semantik

| Parameter | Bedeutung | Effekt bei größerem Wert |
|-----------|-----------|-------------------------|
| `min_duration_on` | Mindestdauer einer Sprachregion | Kurze Äußerungen verworfen → weniger Segmente |
| `min_duration_off` | Max. Pause zum Zusammenführen | Länger Pausen werden überbrückt → weniger, längere Segmente |

## Technischer Ansatz: Monkey-Patching

Da `run_inference_for_experiment` die Segmentierungsparameter nicht direkt
entgegennimmt, wird `InferenceEngine.chunk_video` zur Laufzeit überschrieben
(`patch_avsr_segmentation`). Der Patch ist als Context-Manager implementiert:
nach dem Experiment wird die Original-Methode sauber wiederhergestellt.

## Ergebnis (Vorschau)

**Beste Kombination: min_on=1.0 s, min_off=1.2 s** (E72)

| Konfiguration | WER | Joint Error |
|--------------|-----|-------------|
| E56 (Bugfix-Default, kein Override) | 0.5245 | 0.3384 |
| **E72 (min_on=1.0, min_off=1.2)** | **0.4902** | **0.3213** |
| Δ | **−0.034** | **−0.017** |

E72 liegt damit sogar unter dem Vor-Bugfix-Wert (0.4954) und wird
als finale Konfiguration für `04_dev_final_results` verwendet.

## 1 – GPU-Check & Auswahl

In [1]:
!nvidia-smi

Wed Feb  4 18:35:07 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:01:00.0 Off |                    0 |
| N/A   30C    P0             79W /  500W |   10041MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
import os

# Physische GPU-Auswahl: hier GPU 2 (siehe nvidia-smi)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


## 2 – CUDA-Verifikation

In [3]:
import torch

In [4]:
print("CUDA devices:", torch.cuda.device_count())
print("Device 0 name (sollte A100 sein):", torch.cuda.get_device_name(0))

CUDA devices: 1
Device 0 name (sollte A100 sein): NVIDIA A100-SXM4-80GB


## 3 – Setup: Arbeitsverzeichnis & Imports

In [5]:
import os, sys
import pandas as pd

# Arbeitsverzeichnis auf Repo-Root setzen (Voraussetzung für alle relativen Pfade)
project_baseline_path = "/home/josch080/Projektgruppe/mcorec_baseline"
os.chdir(project_baseline_path)

# Repo-Root in sys.path, damit projektinterne Module importierbar sind
if project_baseline_path not in sys.path:
    sys.path.append(project_baseline_path)

from script.pg_utils_experiments import run_inference_for_experiment, run_eval_and_log, append_eval_results_for_experiments

/home/josch080/Projektgruppe/mcorec_baseline/script/inference.py:22: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## 4 – Monkey-Patch: Segmentierungsparameter zur Laufzeit überschreiben

`run_inference_for_experiment` nimmt `min_duration_on/off` nicht direkt entgegen.
Die Lösung: `InferenceEngine.chunk_video` wird als Context-Manager temporär
durch eine patched Version ersetzt, die die gewünschten Parameter an
`segment_by_asd()` weitergibt. Nach dem `with`-Block wird die Original-Methode
sauber wiederhergestellt – auch bei Exceptions (`finally`-Clause).

In [6]:
from contextlib import contextmanager
import script.inference as inf  # dasselbe Modul das pg_utils_experiments nutzt

@contextmanager
def patch_avsr_segmentation(min_on=None, min_off=None):
    # Context-Manager: überschreibt InferenceEngine.chunk_video für den ASD-Zweig
    # Ermöglicht das Setzen von min_duration_on/off ohne Änderung an pg_utils_experiments
    # Nach dem with-Block wird die Original-Methode wiederhergestellt (auch bei Exceptions)
    orig = inf.InferenceEngine.chunk_video # Original-Methode sichern

    # wenn nix gesetzt -> kein Patch nötig
    if min_on is None and min_off is None:
        yield # Kein Patch nötig → Original-Verhalten
        return

    def patched(self, video_path, asd_path=None, max_length=15):
        if asd_path is not None:
            with open(asd_path, "r") as f:
                asd = inf.json.load(f)

            frames = sorted([int(f) for f in asd.keys()])
            if not frames:
                return []

            min_frame = min(frames)

            # Parameter-Dict für segment_by_asd aufbauen
            params = {"max_chunk_size": max_length}
            if min_on is not None:
                params["min_duration_on"] = float(min_on)
            if min_off is not None:
                params["min_duration_off"] = float(min_off)

            segments_by_frames = inf.segment_by_asd(asd, params)

            # Frame-Indizes in Sekunden konvertieren (25 fps → / 25)
            return [((seg[0] - min_frame) / 25, (seg[-1] - min_frame) / 25) for seg in segments_by_frames]

        # ohne ASD: Originalverhalten
        return orig(self, video_path, asd_path, max_length)

    inf.InferenceEngine.chunk_video = patched
    try:
        yield
    finally:
        inf.InferenceEngine.chunk_video = orig # immer wiederherstellen


def _tag(x: float) -> str:
    # Konvertiert float zu Dateinamen-sicherem String: 1.2 → '1p2'
    return str(x).replace(".", "p")


## 5 – Experiment-Grid (E56–E72)

Beam-Size und max_length sind fixiert (beste Kombination aus `02j_`).
Das 4×4-Grid über `min_duration_on` × `min_duration_off` ergibt
16 Experimente (E57–E72) plus eine Referenz ohne Override (E56).

In [7]:
# Fixe Decoding-Parameter (beste Kombination aus 02j_)
GRID_BEAM = 12
GRID_LEN  = 20

# Grid-Achsen für Segmentierungsparameter (in Sekunden)
MIN_ON_LIST  = [0.4, 0.6, 0.8, 1.0]     # Mindestdauer Sprachregion
MIN_OFF_LIST = [0.5, 0.8, 1.0, 1.2]     # Max. Pause für Gap-Filling

EXPERIMENTS = {}

# E56: Referenz ohne Parameter-Override (Bugfix-Default)
EXPERIMENTS["E56_bugfix_default_bs12_len20"] = {
    "base_model": "avsr_cocktail_finetuned",
    "beam_size": GRID_BEAM,
    "max_length": GRID_LEN,
    "comment": "Bugfix-default segmentation (kein Override von min_duration)",
}

# E57–E72: 4×4-Grid
eid = 57
for min_on in MIN_ON_LIST:
    for min_off in MIN_OFF_LIST:
        exp = f"E{eid}_bugfix_mdOn{_tag(min_on)}_mdOff{_tag(min_off)}_bs{GRID_BEAM}_len{GRID_LEN}"
        EXPERIMENTS[exp] = {
            "base_model": "avsr_cocktail_finetuned",
            "beam_size": GRID_BEAM,
            "max_length": GRID_LEN,
            "min_duration_on": float(min_on),
            "min_duration_off": float(min_off),
            "comment": f"AVSR-Override: min_on={min_on}s, min_off={min_off}s (nur ASD-Chunks)",
        }
        eid += 1

len(EXPERIMENTS), list(EXPERIMENTS.keys())


(17,
 ['E56_bugfix_default_bs12_len20',
  'E57_bugfix_mdOn0p4_mdOff0p5_bs12_len20',
  'E58_bugfix_mdOn0p4_mdOff0p8_bs12_len20',
  'E59_bugfix_mdOn0p4_mdOff1p0_bs12_len20',
  'E60_bugfix_mdOn0p4_mdOff1p2_bs12_len20',
  'E61_bugfix_mdOn0p6_mdOff0p5_bs12_len20',
  'E62_bugfix_mdOn0p6_mdOff0p8_bs12_len20',
  'E63_bugfix_mdOn0p6_mdOff1p0_bs12_len20',
  'E64_bugfix_mdOn0p6_mdOff1p2_bs12_len20',
  'E65_bugfix_mdOn0p8_mdOff0p5_bs12_len20',
  'E66_bugfix_mdOn0p8_mdOff0p8_bs12_len20',
  'E67_bugfix_mdOn0p8_mdOff1p0_bs12_len20',
  'E68_bugfix_mdOn0p8_mdOff1p2_bs12_len20',
  'E69_bugfix_mdOn1p0_mdOff0p5_bs12_len20',
  'E70_bugfix_mdOn1p0_mdOff0p8_bs12_len20',
  'E71_bugfix_mdOn1p0_mdOff1p0_bs12_len20',
  'E72_bugfix_mdOn1p0_mdOff1p2_bs12_len20'])

## 6 – Modell-Definitionen

Nur `avsr_cocktail_finetuned` (BL4) wird in den Experimenten genutzt.

In [8]:
BASE_MODELS = {
    "auto_avsr": {
        "model_type": "auto_avsr",
        "chkpt": "model-bin/auto_avsr/avsr_trlrwlr2lrs3vox2avsp_base.pth",
    },
    "avsr_cocktail": {
        "model_type": "avsr_cocktail",
        "chkpt": "model-bin/avsr_cocktail",
    },
    "avsr_cocktail_finetuned": {
        "model_type": "avsr_cocktail",
        "chkpt": "model-bin/avsr_cocktail_mcorec_finetune",
    },
    "muavic_en": {
        "model_type": "muavic_en",
        "chkpt": "nguyenvulebinh/AV-HuBERT-MuAViC-en",
    },
}


## 7 – Sessions

In [9]:
SESSION_IDS = ["session_40", "session_43", "session_49", "session_50", "session_54"]

## 8 – Inference mit Monkey-Patch

Jedes Experiment wird innerhalb des `patch_avsr_segmentation`-Context-Managers
ausgeführt. Der Patch ist aktiv für genau einen `run_inference_for_experiment`-Aufruf;
danach wird die Original-Methode wiederhergestellt.

CUDA-Cache und Garbage-Collector werden nach jedem Experiment geleert,
um Speicherlecks bei vielen aufeinanderfolgenden GPU-Läufen zu verhindern.

In [10]:
import gc, torch

for sid in SESSION_IDS:
    session_dir = f"data-bin/dev/{sid}"
    print(f"\n########## Starte Grid-Experimente für {sid} ##########")

    for exp_name, cfg in EXPERIMENTS.items():
        min_on  = cfg.get("min_duration_on", None)
        min_off = cfg.get("min_duration_off", None)

        # Patch ist nur innerhalb dieses with-Blocks aktiv
        with patch_avsr_segmentation(min_on=min_on, min_off=min_off):
            run_inference_for_experiment(
                exp_name=exp_name,
                base_models=BASE_MODELS,
                experiments=EXPERIMENTS,
                session_dir=session_dir,
            )

        # Speicher nach jedem Experiment freigeben
        torch.cuda.empty_cache()
        gc.collect()



########## Starte Grid-Experimente für session_40 ##########

Starte Inference für Experiment: E56_bugfix_default_bs12_len20
  base_model      = avsr_cocktail_finetuned
  model_type      = avsr_cocktail
  checkpoint_path = model-bin/avsr_cocktail_mcorec_finetune
  beam_size       = 12
  max_length      = 20
  output_dir_name = output_E56_bugfix_default_bs12_len20
  session_dir     = data-bin/dev/session_40
  comment         = Bugfix-default segmentation (kein Override von min_duration)
Loading avsr_cocktail model...


/home/josch080/Projektgruppe/mcorec_train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model from model-bin/avsr_cocktail_mcorec_finetune
avsr_cocktail model loaded successfully!
Inferring 1 sessions using avsr_cocktail model
Processing session session_40


Processing speakers:   0%|          | 0/6 [00:00<?, ?it/s]


Processing speakers:  17%|█▋        | 1/6 [00:42<03:34, 42.82s/it]

...


Processing speaker spk_0 track 1 of 2: 100%|██████████| 26/26 [00:48<00:00,  1.85s/it]

Processing speakers:  20%|██        | 1/5 [01:02<04:08, 62.24s/it]


Processing speakers:  40%|████      | 2/5 [02:21<03:35, 71.98s/it]


Processing speakers:  60%|██████    | 3/5 [03:16<02:09, 64.56s/it]


Processing speakers:  80%|████████  | 4/5 [04:42<01:13, 73.11s/it]


Processing speakers: 100%|██████████| 5/5 [05:28<00:00, 65.65s/it]


## 9 – Evaluation & Aggregation

Separate CSV (`results_grid_segmentation_by_session.csv`) –
nicht die gemeinsame Experiment-CSV, da dieses Grid andere Parameter hat.

In [12]:
df_by_session = append_eval_results_for_experiments(
    experiments=EXPERIMENTS,
    session_ids=SESSION_IDS,
    target_csv="results_grid_segmentation_by_session.csv",
    session_dir_template="data-bin/dev/{sid}",   # ggf. anpassen
    label_dir="labels",
)


########## Evaluate für session_40 ##########
Starte Evaluate: /home/josch080/Projektgruppe/mcorec_train/bin/python script/evaluate.py --session_dir data-bin/dev/session_40 --output_dir_name output_ --label_dir_name labels
Evaluating 1 sessions

=== Evaluating session session_40 ===

--- Evaluating output dir: output_E01_bs4_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.564, 'spk_1': 0.4281, 'spk_2': 0.5576, 'spk_3': 0.4283, 'spk_4': 0.4793, 'spk_5': 0.4189}
Speaker clustering F1 score: {'spk_0': 1.0, 'spk_1': 1.0, 'spk_2': 1.0, 'spk_3': 1.0, 'spk_4': 1.0, 'spk_5': 1.0}
Joint ASR-Clustering Error Rate: {'spk_0': 0.282, 'spk_1': 0.21405, 'spk_2': 0.2788, 'spk_3': 0.21415, 'spk_4': 0.23965, 'spk_5': 0.20945}

--- Evaluating output dir: output_E02_bs8_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.561, 'spk_1': 0.4312, 'spk_2': 0.5506, 'spk_3': 0.4283, 'spk_4': 0.5041, 'spk_5': 0.4189}
Speaker clustering F1 score: {'spk_0': 1.0, 

/home/josch080/Projektgruppe/mcorec_baseline/script/pg_utils_experiments.py:381: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_df], ignore_index=True)


## 10 – Ergebnisanalyse: Top-15-Konfigurationen

In [13]:
# Mittelwert über alle 5 Sessions pro Experiment
summary = (df_by_session
           .groupby("model", as_index=False)[["avg_conv_f1","avg_speaker_wer","avg_joint_error"]]
           .mean()
           .sort_values("avg_speaker_wer")) # aufsteigend: beste WER oben

# Ergebnisse persistieren
summary.to_csv("results_grid_segmentation_summary.csv", index=False)

display(summary.head(15)) # Top-15 nach WER
print("Saved:",
      "results_grid_segmentation_by_session.csv",
      "results_grid_segmentation_summary.csv")


,model,avg_conv_f1,avg_speaker_wer,avg_joint_error
16,E72_bugfix_mdOn1p0_mdOff1p2_bs12_len20,0.847619,0.490223,0.321304
4,E60_bugfix_mdOn0p4_mdOff1p2_bs12_len20,0.847619,0.490475,0.321430
12,E68_bugfix_mdOn0p8_mdOff1p2_bs12_len20,0.847619,0.490515,0.321449
8,E64_bugfix_mdOn0p6_mdOff1p2_bs12_len20,0.847619,0.490828,0.321606
15,E71_bugfix_mdOn1p0_mdOff1p0_bs12_len20,0.847619,0.495416,0.323900
11,E67_bugfix_mdOn0p8_mdOff1p0_bs12_len20,0.847619,0.495567,0.323976
3,E59_bugfix_mdOn0p4_mdOff1p0_bs12_len20,0.847619,0.495808,0.324096
7,E63_bugfix_mdOn0p6_mdOff1p0_bs12_len20,0.847619,0.496161,0.324272
14,E70_bugfix_mdOn1p0_mdOff0p8_bs12_len20,0.847619,0.515067,0.333725
10,E66_bugfix_mdOn0p8_mdOff0p8_bs12_len20,0.847619,0.515175,0.333780


Saved: results_grid_segmentation_by_session.csv results_grid_segmentation_summary.csv


## 11 – Bestes Modell & Vergleich zur Default-Konfiguration

In [15]:
# Beste WER (summary ist nach avg_speaker_wer sortiert)
best = summary.iloc[0]
print("Best:", best["model"],
      "WER=", best["avg_speaker_wer"],
      "Joint=", best["avg_joint_error"])

# Vergleich zur Default-Konfiguration (Bugfix ohne Override)
DEFAULT_MODEL = "E56_bugfix_default_bs12_len20"  # falls bei dir anders heißt, hier anpassen

row_default = summary[summary["model"] == DEFAULT_MODEL]
if row_default.empty:
    print(f"\nWARN: Default '{DEFAULT_MODEL}' nicht in summary gefunden.")
    print("Verfügbare Modelle (Beispiele):", list(summary["model"].head(10)))
else:
    d = row_default.iloc[0]

     # Δ: negativ = Verbesserung (niedrigere WER ist besser)
    delta_wer = best["avg_speaker_wer"] - d["avg_speaker_wer"]
    delta_joint = best["avg_joint_error"] - d["avg_joint_error"]

    rel_delta_wer = (delta_wer / d["avg_speaker_wer"]) * 100
    rel_delta_joint = (delta_joint / d["avg_joint_error"]) * 100

    print(f"\nVergleich zu Default: {DEFAULT_MODEL}")
    print(f"  Default  WER={d['avg_speaker_wer']:.6f}  Joint={d['avg_joint_error']:.6f}")
    print(f"  Best     WER={best['avg_speaker_wer']:.6f}  Joint={best['avg_joint_error']:.6f}")
    print(f"  ΔWER     {delta_wer:+.6f}  ({rel_delta_wer:+.3f}%)")
    print(f"  ΔJoint   {delta_joint:+.6f}  ({rel_delta_joint:+.3f}%)")


Best: E72_bugfix_mdOn1p0_mdOff1p2_bs12_len20 WER= 0.49022333333333334 Joint= 0.32130366666666665

Vergleich zu Default: E56_bugfix_default_bs12_len20
  Default  WER=0.524484  Joint=0.338434
  Best     WER=0.490223  Joint=0.321304
  ΔWER     -0.034261  (-6.532%)
  ΔJoint   -0.017130  (-5.062%)


## 12 – Interpretation

| Konfiguration | WER | Joint Error | Δ WER vs. E56 |
|--------------|-----|-------------|---------------|
| E56 (Bugfix-Default, beam=12, len=20) | 0.5245 | 0.3384 | – |
| **E72 (min_on=1.0, min_off=1.2)** | **0.4902** | **0.3213** | **−0.034** |
| Vor-Bugfix-Referenz (E09, beam=12, len=20) | 0.4954 | 0.3239 | – |

*(Werte auf dem 5-Session-Subset; auf allen 25 Dev-Sessions: WER 0.4943, siehe `04_dev_final_results`)*

**E72 übertrifft sogar das Vor-Bugfix-Ergebnis** (0.4902 vs. 0.4954).

### Warum helfen größere min_duration_on/off?

- **min_on=1.0 s:** Kurze Sprachfragmente (<1 s) werden verworfen.
  Das reduziert Rauschen und unvollständige Äußerungen im Input.
- **min_off=1.2 s:** Pausen bis 1.2 s werden überbrückt – Sätze
  mit kurzen Sprechpausen bleiben zusammen, was den Decoder-Kontext verbessert.

**Diese Konfiguration (min_on=1.0, min_off=1.2, beam=12, len=20)**
wird als finale Konfiguration für `04_dev_final_results` und
`04_eval_final_results` verwendet.
